# Import Data

In [1]:
import pandas as pd
train_df = pd.read_csv('../../input/feedback-prize-english-language-learning/train.csv')


model_path_list = [
    '../../01_Baseline/exp/result/01_v1_24/oof_df.csv', # deberta-base
    '../../01_Baseline/exp/result/01_v1_25/oof_df.csv', # deberta-large
    '../../01_Baseline/exp/result/01_v1_27/oof_df.csv', # deberta-v3-base

    '../../01_Baseline/exp/result/01_v1_43/oof_df.csv', # deberta-v3-large
    '../../01_Baseline/exp/result/01_v1_44/oof_df.csv', # roberta-base
    '../../01_Baseline/exp/result/01_v1_45/oof_df.csv', # roberta-large
    '../../01_Baseline/exp/result/01_v1_47/oof_df.csv', # electra-base
    '../../01_Baseline/exp/result/01_v1_49/oof_df.csv', # deepset/roberta-base-squad2
    '../../01_Baseline/exp/result/01_v1_50/oof_df.csv', # distilroberta-base
    
    '../../01_Baseline/exp/result/01_v1_54/oof_df.csv', # electra-large
    '../../01_Baseline/exp/result/01_v1_55/oof_df.csv', # deepset/roberta-large-squad2
    '../../01_Baseline/exp/result/01_v1_56/oof_df.csv', # xlm-roberta-large
    '../../01_Baseline/exp/result/01_v1_59/oof_df.csv', # bert-large-cased-whole-word-masking
    '../../01_Baseline/exp/result/01_v1_60/oof_df.csv', # bert-large-cased
    '../../07_Embeddings/exp/result/07_v1_01/oof_df_svr_notebook_v03.csv', # svr
    
]

oof_df_list = [
    pd.read_csv(model_path) for model_path in model_path_list
]

for oof_df in oof_df_list:
    oof_df = train_df[['text_id']].merge(oof_df, how='left', on='text_id')
    
num_models = len(model_path_list)

TARGET_COLS = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

stack_df = oof_df_list[0][['text_id']]
for i in range(num_models):
    oof_df = oof_df_list[i]
    for col in TARGET_COLS:
        stack_df[f'{col}_{i}'] = oof_df[col].values
        
        
train_df = train_df[['text_id']+TARGET_COLS].merge(stack_df, on='text_id', how='left')

/home/takesako/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [2]:
train_df.head()

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_0,syntax_0,vocabulary_0,...,vocabulary_13,phraseology_13,grammar_13,conventions_13,cohesion_14,syntax_14,vocabulary_14,phraseology_14,grammar_14,conventions_14
0,0016926B079C,3.5,3.5,3.0,3.0,4.0,3.0,3.019583,3.031228,3.068353,...,3.136100,3.054777,3.188607,2.866543,3.090008,2.942046,3.104937,3.210109,3.330849,2.864747
1,0022683E9EA5,2.5,2.5,3.0,2.0,2.0,2.5,2.660616,2.499767,2.859407,...,2.964505,2.882491,2.566260,2.645195,2.544430,2.470382,2.831750,2.606420,2.401873,2.524009
2,00299B378633,3.0,3.5,3.0,3.0,3.0,2.5,2.796222,2.757307,2.909814,...,3.095113,3.084174,3.067044,3.060621,2.797657,2.960948,3.124284,2.961082,2.972845,3.013185
3,003885A45F42,4.5,4.5,4.5,4.5,4.0,5.0,3.767042,3.684418,3.786009,...,3.787250,3.746300,3.615203,3.738615,3.569966,3.596947,3.599233,3.713636,3.782764,3.773484
4,0049B1DF5CCC,2.5,3.0,3.0,3.0,2.5,2.5,2.513415,2.274921,2.641665,...,2.567789,2.513104,2.422624,2.236006,2.622300,2.448069,2.731655,2.892202,2.687519,2.394289


# Features

In [3]:
feature_cols = [col for col in stack_df.columns if col!='text_id']

# Data Split

In [4]:
from os.path import join as opj

class args:
    fold_path = '../../00_EDA/00_v1_02/result/'
    num_fold = 5
    
fold_path = args.fold_path
import joblib
print('load folds...')
trn_ids_list = joblib.load(opj(fold_path,f'trn_ids_list.joblib'))
val_ids_list = joblib.load(opj(fold_path,f'val_ids_list.joblib'))

load folds...


# Stacking

In [5]:
import matplotlib.pyplot as plt

def show_feature_importance(model):
    feature_importance = model.get_feature_importance()
    fi_df = pd.DataFrame({'importance':feature_importance, 'features':feature_cols})
    fi_df = fi_df.sort_values('importance', ascending=True)

    plt.figure(figsize=(12, 6))
    plt.barh(range(len(fi_df)),
            fi_df['importance'].values,
            tick_label=fi_df['features'].values)
    plt.xlabel('importance')
    plt.ylabel('features')
    plt.grid()
    plt.show()

In [6]:
import catboost
params = {
    'random_state':45, 
    'n_estimators':20000,
    'loss_function':'RMSE',
}

import numpy as np
def calc_metric(pred, gt):
    '''
    pred : (num_data, num_labels)
    gt : (num_data, num_labels)
    '''
    score = np.sqrt(np.mean((pred - gt)**2, axis=0))
    score = score.mean()
    return score

In [7]:
from sklearn.metrics import log_loss
import pickle
import os
os.makedirs('./result', exist_ok=True)

cat_features = []


score_list = []
oof_df = []

for fold in range(args.num_fold):
    trn_df = train_df[train_df['text_id'].isin(trn_ids_list[fold])].reset_index(drop=True)
    val_df = train_df[train_df['text_id'].isin(val_ids_list[fold])].reset_index(drop=True)
    preds = np.zeros((len(val_df), len(TARGET_COLS)))

    for i,col in enumerate(TARGET_COLS):
        tmp_feature_cols = [c for c in feature_cols if c.startswith(col)]
        pool_trn = catboost.Pool(trn_df[tmp_feature_cols].values, label=trn_df[col].values, cat_features=cat_features)
        pool_val = catboost.Pool(val_df[tmp_feature_cols].values, label=val_df[col].values, cat_features=cat_features)

        model = catboost.CatBoostRegressor(**params)
        model.fit(pool_trn, 
                  eval_set=[pool_val], 
                  verbose=0, 
                  early_stopping_rounds=100,
                 )

        pred = model.predict(pool_val)
        preds[:,i] = pred
        # save model
        joblib.dump(model, f'./result/cat_fold{fold}_{i}.joblib')
        # plot
        #show_feature_importance(model)
    
    target = val_df[TARGET_COLS].values
    score = calc_metric(preds, target)
    print('fold{} : CV={:.4f}'.format(fold, score))
    score_list.append(score)
    for i,col in enumerate(TARGET_COLS):
        val_df[f'oof_{col}'] = preds[:,i]
    oof_df.append(val_df)
    
CV = sum(score_list) / len(score_list)
print('CV={:.4f}'.format(CV))

fold0 : CV=0.4482
fold1 : CV=0.4439
fold2 : CV=0.4436
fold3 : CV=0.4527
fold4 : CV=0.4528
CV=0.4482


In [8]:
oof_df = pd.concat(oof_df).reset_index(drop=True)
oof_df = train_df[['text_id']].merge(oof_df, on='text_id', how='left')
oof_df.to_csv(f'./result/oof_cat.csv', index=False)
print(oof_df.shape)
oof_df.head()

(3911, 103)


,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_0,syntax_0,vocabulary_0,...,vocabulary_14,phraseology_14,grammar_14,conventions_14,oof_cohesion,oof_syntax,oof_vocabulary,oof_phraseology,oof_grammar,oof_conventions
0,0016926B079C,3.5,3.5,3.0,3.0,4.0,3.0,3.019583,3.031228,3.068353,...,3.104937,3.210109,3.330849,2.864747,2.956481,2.905225,3.056159,3.104861,3.128061,2.844259
1,0022683E9EA5,2.5,2.5,3.0,2.0,2.0,2.5,2.660616,2.499767,2.859407,...,2.831750,2.606420,2.401873,2.524009,2.606351,2.517641,2.860775,2.701986,2.499664,2.469643
2,00299B378633,3.0,3.5,3.0,3.0,3.0,2.5,2.796222,2.757307,2.909814,...,3.124284,2.961082,2.972845,3.013185,2.816414,2.933338,3.080712,2.957589,2.973324,2.987702
3,003885A45F42,4.5,4.5,4.5,4.5,4.0,5.0,3.767042,3.684418,3.786009,...,3.599233,3.713636,3.782764,3.773484,3.620697,3.557979,3.651067,3.713097,3.670657,3.783195
4,0049B1DF5CCC,2.5,3.0,3.0,3.0,2.5,2.5,2.513415,2.274921,2.641665,...,2.731655,2.892202,2.687519,2.394289,2.629664,2.355132,2.787378,2.620617,2.505730,2.398116


In [9]:
cols = [col for col in oof_df.columns if col.startswith('oof_')]

oof_score = calc_metric(oof_df[cols].values, oof_df[TARGET_COLS].values)
print('oof={:.4f}'.format(oof_score))

oof=0.4483
